In [4]:
import requests
import folium
import pandas as pd
from folium import IFrame

# Fetch the data from the API for COPD
url_copd = 'http://127.0.0.1:9090/airquality/copd'
response_copd = requests.get(url_copd)
data_copd = response_copd.json()

# Convert JSON data to a DataFrame
df_copd = pd.json_normalize(data_copd)

# Fetch the data from the API for RSD
url_rsd = 'http://127.0.0.1:9090/airquality/rsd'
response_rsd = requests.get(url_rsd)
data_rsd = response_rsd.json()

# Convert JSON data to a DataFrame
df_rsd = pd.json_normalize(data_rsd)

# Find the maximum values for RSD admissions
max_total_rsd = df_rsd['average_total_rsd_admissions_num'].max()
max_male_rsd = df_rsd['average_male_rsd_admissions_num'].max()
max_female_rsd = df_rsd['average_female_rsd_admissions_num'].max()

# Create a Folium map centered on a specific location
m = folium.Map(width=900, height=600, location=[-36.5115, 144.1652], zoom_start=7, tiles='OpenStreetMap')

# Define a function to create styled HTML for the popups
def create_popup_html(row, admissions_type):
    html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <h4>{row['site_name']}</h4>
        <p><strong>Health Advice:</strong> <span style="color: {row['health_advice_color']};">{row['health_advice']}</span></p>
        <p><strong>Value:</strong> {row['value']} {row['unit']}</p>
        <p><strong>Average {admissions_type} Admissions:</strong></p>
        <ul>
            <li>Total: {int(row[f'average_total_{admissions_type}_admissions_num'])} people</li>
            <li>Male: {int(row[f'average_male_{admissions_type}_admissions_num'])} people</li>
            <li>Female: {int(row[f'average_female_{admissions_type}_admissions_num'])} people</li>
        </ul>
    </div>
    """
    return html

# Add markers for COPD data
for index, row in df_copd.iterrows():
    popup_html = create_popup_html(row, "copd")
    iframe = IFrame(popup_html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    
    icon_color = "blue"
    if row['average_total_copd_admissions_num'] == df_copd['average_total_copd_admissions_num'].max():
        icon_color = "red"
    elif row['average_male_copd_admissions_num'] == df_copd['average_male_copd_admissions_num'].max():
        icon_color = "green"
    elif row['average_female_copd_admissions_num'] == df_copd['average_female_copd_admissions_num'].max():
        icon_color = "purple"
    
    folium.Marker(
        location=[row['site_location.lat'], row['site_location.lon']],
        popup=popup,
        tooltip=f"<strong>{row['site_name']}</strong>",
        icon=folium.Icon(color=icon_color, icon="info-sign"),
    ).add_to(m)

# Add markers for RSD data
for index, row in df_rsd.iterrows():
    popup_html = create_popup_html(row, "rsd")
    iframe = IFrame(popup_html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    
    icon_color = "blue"
    if row['average_total_rsd_admissions_num'] == max_total_rsd:
        icon_color = "red"
    elif row['average_male_rsd_admissions_num'] == max_male_rsd:
        icon_color = "green"
    elif row['average_female_rsd_admissions_num'] == max_female_rsd:
        icon_color = "purple"
    
    folium.Marker(
        location=[row['site_location.lat'], row['site_location.lon']],
        popup=popup,
        tooltip=f"<strong>{row['site_name']}</strong>",
        icon=folium.Icon(color=icon_color, icon="info-sign"),
    ).add_to(m)

# Display the map
m
